# 1. Subject averaging.

## Goals:

Subjects viewed two slightly different types of video as described in the HCP 12000 reference manual (https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf). Rather than averaging the data across all subjects, it is better to do due dilligence and average them depending on the specific video that they viewed (even if the differences are minor). 

This allows us to investigate the agreement of the parameters estimated from two across-subject folds.

I want to determine which subject viewed what video, average accordingly and then save in the appropriate format.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2
from cfhcpy.base import AnalysisBase
from funcs import HCP_subject
from tqdm import tqdm
import numpy as np
from collections import Counter
from joblib import Parallel, delayed

import nibabel as nib
import os
import cifti
from cfhcpy.surf_utils import split_cii

/tank/hedger/software/hcp_movie/cfhcpy/base.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/tank/hedger/software/hcp_movie/cfhcpy/fit_utils.py:22: UserWarning: 

 | Using Nistats with Nilearn versions >= 0.7.0 is redundant and potentially conflicting.
 | Nilearn versions 0.7.0 and up offer all the functionality of Nistats as well the latest features and fixes.
 | We strongly recommend uninstalling Nistats and using Nilearn's stats & reporting modules.

  from nistats.hemodynamic_models import spm_hrf
/tank/hedger/software/anaconda3/envs/p3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.ridge module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported fro

Start an analysis base with an arbitrary subject, just so we have access to all the paths etc.

In [2]:
ab = AnalysisBase()
subno=114823

In [3]:
ab.startup(subject=str(subno), experiment_id='movie', yaml_file='/tank/hedger/software/hcp_movie/config.yml')

Starting analysis of subject 114823 on romulus with settings 
{
 "identifier": "node230",
 "base_dir": "/scratch/2019/visual/hcp_{experiment}/",
 "code_dir": "/tank/hedger/scripts/HCP_tonotopy",
 "threads": 40
}


Go through all the subjects, use their CSV file to return the video they watched. Dump this in a list.

In [ ]:
movtype=[]
dpaths=[]

for sub in tqdm(ab.full_data_subjects):
    tbase = AnalysisBase()
    tbase.startup(subject=str(sub), experiment_id='movie', yaml_file='/tank/hedger/software/hcp_movie/config.yml')
    mysub=HCP_subject(tbase)
    movtype.append(mysub.vidprefix)
    mysub.get_data_paths()
    dpaths.append(mysub.dpaths)
    tbase=None

Count the number of participants that viewed each video.

In [8]:
Counter(movtype).values()

dict_values([132, 42])

It would seem that most viewed the new video. Get the indices of the participants that viewed each video.

In [9]:
earlyvidsubs= [i for i, s in enumerate(movtype) if 'Pre_20140821' in s]
latevidsubs= [i for i, s in enumerate(movtype) if 'Post_20140821' in s]

Define function for reading in the data for a specific run/movie

In [14]:
def read_array(dpaths,s,movind):
    data=nib.load(dpaths[s][movind])
    darray=np.array(data.get_data())
    return darray

We want to output the early and late subjects to different directories. Do this in my own space so as to not disrupt the original folder. 

In [16]:
earlydir='/tank/hedger/DATA/HCP_temp/early'
latedir='/tank/hedger/DATA/HCP_temp/late'

We might as well preserve the same kind of filename so that all of the HCP_movie functionality will still work.

In [25]:
PEDS=['AP','PA','PA','AP']

Define a function for creating the mean data.

In [31]:
def create_mean_data(dpaths,subs,movienum,outputdir,save):
    
    sub_readin=Parallel(n_jobs=20,verbose=1)(delayed(read_array)(dpaths,s,movienum)  for s in subs) #Read in all the data
    sub_array=np.array(sub_readin) # Make into big array
    sub_meandat=np.mean(sub_array,0) # Take mean.
    
    if save==True:
        
        tps=np.array(range(ab.experiment_dict['run_durations'][movienum])).astype('str') # The cifti writing seems to require us to give the number of timepoints
        
        fname=os.path.join(outputdir,'tfMRI_{experiment_id}{run}_{PED}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc.nii'.format(experiment_id='MOVIE',run=str(ab.experiment_dict['runs'][movienum]),PED=PEDS[movienum]))        
        
        
        cifti.write(fname, sub_meandat,(cifti.Scalar.from_names(tps),ab.cifti_brain_model))
    
    return sub_meandat,fname

Define a function for splitting the data into the two cortical surfaces using workbench commands.

In [ ]:

def split_dfile(f2split):
        # Summons workbench to split a data file
        nfileL,nfileR=f2split[:-4]+'_L.gii',f2split[:-4]+'_R.gii'
        if not os.path.isfile(nfileL):
            result=split_cii(fn=f2split,workbench_split_command=ab.workbench_split_command,resample=False)
        else:
            print(f'split files already exist subject {ab.subject}','Skipping..')
        
        return nfileL,nfileR

Now go though the early and late subjects, save the mean volume for each movie and then split it into surface giftis.

Break this task down and do it for each run. It is quite memory intensive given the size of the arrays we are reading in.

In [32]:
x=create_mean_data(dpaths,earlyvidsubs,0,earlydir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  42 out of  42 | elapsed:   50.6s finished


('/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE1_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE1_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [33]:
x=create_mean_data(dpaths,earlyvidsubs,1,earlydir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  42 out of  42 | elapsed:   53.6s finished


('/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE2_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE2_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [34]:
x=create_mean_data(dpaths,earlyvidsubs,2,earlydir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  42 out of  42 | elapsed:   57.8s finished


('/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE3_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE3_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [35]:
x=create_mean_data(dpaths,earlyvidsubs,3,earlydir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  42 out of  42 | elapsed:   50.9s finished


('/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE4_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/early/tfMRI_MOVIE4_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [38]:
x=None

In [37]:
x=create_mean_data(dpaths,latevidsubs,0,latedir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   16.1s
[Parallel(n_jobs=20)]: Done 132 out of 132 | elapsed:  2.6min finished


('/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE1_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE1_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [39]:
x=create_mean_data(dpaths,latevidsubs,1,latedir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   16.8s
[Parallel(n_jobs=20)]: Done 132 out of 132 | elapsed:  2.7min finished


('/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE2_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE2_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [41]:
x=create_mean_data(dpaths,latevidsubs,2,latedir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   15.4s
[Parallel(n_jobs=20)]: Done 132 out of 132 | elapsed:  2.5min finished


('/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE3_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE3_PA_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')

In [43]:
x=create_mean_data(dpaths,latevidsubs,3,latedir,True)
mysub.split_dfile(x[1])

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   15.7s
[Parallel(n_jobs=20)]: Done 132 out of 132 | elapsed:  2.5min finished


('/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE4_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_L.gii',
 '/tank/hedger/DATA/HCP_temp/late/tfMRI_MOVIE4_AP_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries_sg_psc_R.gii')